In [ ]:
# https://www.tensorflow.org/tutorials/layers
# Импорт библиотек
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Импорт библиотеки для работы с массивами и тензорфлоу
import numpy as np
import tensorflow as tf

# Настройка выводимой в логах информации, аналог vebrose
tf.logging.set_verbosity(tf.logging.INFO)

def cnn_model_fn(features, labels, mode): # Построение нейронной сети
    
  # Входной слой, размер уменьшен до 32х32, как у CIFAR-10
  input_layer = tf.reshape(features["x"], [-1, 32, 32, 1])

  # Сверточный слой 1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)

  # Слой пулинга 1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Сверточный слой 2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)
  
  # Слой пулинга 2
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Сверточный слой 3
  conv3 = tf.layers.conv2d(
      inputs=pool2,
      filters=128,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)
  
  # Слой пулинга 3
  pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)
 
  # Разверточный слой 1
  deconv1 = tf.layers.deconv2d(
      inputs=pool3,
      filters=64,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)
  
  # Слой анпулинга 1
  pre_unpool1 = tf.layers.max_pooling2d(inputs=deconv1, pool_size=[2, 2], strides=2)
  unpool1 = unpool(pre_unpool1, ind, kernel_size=(1, 2, 2, 1), scope='unpool')
    
  # Разверточный слой 2
  deconv2 = tf.layers.deconv2d(
      inputs=unpool1,
      filters=32,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)
  
  # Слой пулинга 3
  pre_unpool2 = tf.layers.max_pooling2d(inputs=deconv2, pool_size=[2, 2], strides=2)
  unpool2 = unpool(pre_unpool2, ind, kernel_size=(1, 2, 2, 1), scope='unpool')
 

  # Изменение размерности входящего слоя, аналог Flatten()
  unpool2_flat = tf.reshape(unpool2, [-1, 7 * 7 * 64])
    
  # Полносвязный слой и дропаут
  dense = tf.layers.dense(inputs=unpool2_flat, units=128, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Слой-классификатор с 10 нейронами для 10 классов
  logits = tf.layers.dense(inputs=dropout, units=10)

  # Формирование результатов
  predictions = {
      # Определение класса для каждого объекта
      "classes": tf.argmax(input=logits, axis=1),
      # 
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Для обеих моделей подсчитваем потери методом категориальной кросс энтропии
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Настройка оптимизатора для модели, стохастический градиентный спуск
  if mode == tf.estimator.ModeKeys.TRAIN: # Если набор данных - тренировочный
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001) # Тренируем сеть с использованием SGD
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Добавление метрики 
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

# Слой анпулинга, взято из https://github.com/tensorflow/tensorflow/issues/2169#issuecomment-216607417
def unpool(pool, ind, ksize=(1, 2, 2, 1), scope='unpool'):
    """
       Unpooling layer after max_pool_with_argmax.
       Args:
           pool:   max pooled output tensor
           ind:      argmax indices (produced by tf.nn.max_pool_with_argmax)
           ksize:     ksize is the same as for the pool
       Return:
           unpooled:    unpooling tensor
    """
    with tf.variable_scope(scope):
        pooled_shape = pool.get_shape().as_list()

        flatten_ind = tf.reshape(ind, (pooled_shape[0], pooled_shape[1] * pooled_shape[2] * pooled_shape[3]))
        # sparse indices to dense ones_like matrics
        one_hot_ind = tf.one_hot(flatten_ind,  pooled_shape[1] * ksize[1] * pooled_shape[2] * ksize[2] * pooled_shape[3], on_value=1., off_value=0., axis=-1)
        one_hot_ind = tf.reduce_sum(one_hot_ind, axis=1)
        one_like_mask = tf.reshape(one_hot_ind, (pooled_shape[0], pooled_shape[1] * ksize[1], pooled_shape[2] * ksize[2], pooled_shape[3]))
        # resize input array to the output size by nearest neighbor
        img = tf.image.resize_nearest_neighbor(pool, [pooled_shape[1] * ksize[1], pooled_shape[2] * ksize[2]])
        unpooled = tf.multiply(img, tf.cast(one_like_mask, img.dtype))
        return unpooled

def main():
  # Загружаем данные для обучения и оценки
  dataset = tf.contrib.learn.datasets.load_dataset("D://KT_dataset/cifarpascal")
  train_data = dataset.train.images # массив numpy для данных обучения
  train_labels = np.asarray(dataset.train.labels, dtype=np.int32) # Приводим к целочисленным значениям
  eval_data = dataset.test.images # массив numpy для данных оценки
  eval_labels = np.asarray(dataset.test.labels, dtype=np.int32) # Приводим к целочисленным значениям
    
  # Добавляем Estimator - класс в TensorFlow для высокоуровнего обучения и оценки моделей
  classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir="/testmodel/") 

  # Периодическое занесение данных в лог
  tensors_to_log = {"probabilities": "softmax_tensor"}
  logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

  # Общие настройки данных для обучения
  train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=35,
    shuffle=True)
  classifier.train(
    input_fn=train_input_fn,
    steps=20000,
    hooks=[logging_hook])

  # Общие настройки данных для обучения
  eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)
  print(eval_results)

    
if __name__ == '__main__':
  #tf.app.run()
  main()